# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
- Write queries that make use of various types of Joins
- Join tables using foreign keys

## CRM Schema

In almost all cases, rather then just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same Customer Relationship Management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database **data.sqlite**.

In [2]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Display the names of all the employees in Boston.
Hint: join the employees and offices tables.

In [3]:
query = """
        SELECT firstName, lastName
        FROM employees
        INNER JOIN offices
        USING(officeCode)
        WHERE city = 'Boston'
        """

df = pd.DataFrame(c.execute(query).fetchall())
df.columns = [x[0] for x in c.description]
df

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [4]:
query = """
        SELECT COUNT(employeeNumber) as num_employees, officeCode
        FROM offices
        LEFT JOIN employees
        USING(officeCode)
        GROUP BY officeCode
        HAVING num_employees = 0;
        """
df = pd.DataFrame(c.execute(query).fetchall())
# df.columns = [x[0] for x in c.description]
df

""


## Write 3 Questions of your own and answer them

In [5]:
# Who is the top 5 customers by amount paid?number of orders and payments?

query = """
        SELECT c.contactLastName, c.contactFirstName, SUM(p.amount) as total_spent, COUNT(o.orderNumber) as total_orders
        FROM customers c
        JOIN payments p
        USING(customerNumber)
        JOIN orders o
        USING(customerNumber)
        GROUP BY c.contactLastName, c.contactFirstName
        ORDER BY total_spent DESC
        """
df = pd.DataFrame(c.execute(query).fetchmany(5))
df.columns = [x[0] for x in c.description]
df

,contactLastName,contactFirstName,total_spent,total_orders
0,Freyre,Diego,18609213.48,338
1,Nelson,Susan,9931200.08,153
2,Ferguson,Peter,902925.35,20
3,Natividad,Eric,781255.15,20
4,Graham,Mike,773110.40,20


In [17]:
# Your code here
import pandas as pd
query = """
        SELECT  c.contactLastName, 
                c.contactFirstName, 
                SUM(od.quantityOrdered) AS num_ordered, 
                (od.quantityOrdered * od.priceEach) AS total_amount_ordered, 
                pl.productLine
        FROM customers c
        JOIN orders o
        USING(customerNumber)
        JOIN orderdetails od
        USING(orderNumber)
        JOIN products p
        USING(productCode)
        JOIN productlines pl
        USING(productLine)
        GROUP BY c.contactLastName, c.contactFirstName, pl.productLine
        ORDER BY total_amount_ordered
    
        """

df = pd.DataFrame(c.execute(query).fetchall())
df.columns = [x[0] for x in c.description]
df

,contactLastName,contactFirstName,num_ordered,total_amount_ordered,productLine
0,Huxley,Adrian,739,529.35,Vintage Cars
1,Frick,Sue,556,546.66,Vintage Cars
2,Lebihan,Laurence,156,557.60,Vintage Cars
3,Shimamura,Akiko,231,615.45,Vintage Cars
4,Taylor,Sue,64,653.52,Classic Cars
5,McRoy,Sarah,129,662.90,Classic Cars
6,Frick,Sue,450,834.20,Planes
7,King,Jean,318,848.54,Vintage Cars
8,Roel,José Pedro,70,856.80,Vintage Cars
9,O'Hara,Anna,220,870.96,Vintage Cars


In [7]:
# Your code here

In [9]:
# Your code here

## Level Up: Display the names of every individual product that each employee has sold

In [26]:
query = """
        SELECT e.firstName || " " || e.lastName as customerName, p.productName
        FROM employees e
        INNER JOIN customers c
        ON e.employeeNumber = c.salesRepEmployeeNumber
        INNER JOIN orders o
        ON o.customerNumber = c.customerNumber
        INNER JOIN orderdetails od
        ON od.orderNumber = o.orderNumber
        INNER JOIN products p
        ON p.productCode = od.productCode
        """
df = pd.DataFrame(c.execute(query).fetchall())
df.columns = [x[0] for x in c.description]
df.set_index('customerName')
# df

,productName
customerName,
Leslie Jennings,1958 Setra Bus
Leslie Jennings,1940 Ford Pickup Truck
Leslie Jennings,1939 Cadillac Limousine
Leslie Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
Leslie Jennings,1968 Ford Mustang
Leslie Jennings,1968 Dodge Charger
Leslie Jennings,1970 Plymouth Hemi Cuda
Leslie Jennings,1969 Dodge Charger
Leslie Jennings,1948 Porsche 356-A Roadster


## Level Up: Display the Number of Products each employee has sold

In [10]:
query = """
        SELECT e.firstName, e.lastName, sum(od.quantityOrdered) as num_products_sold
        FROM employees e
        INNER JOIN customers c
        ON e.employeeNumber = c.salesRepEmployeeNumber
        INNER JOIN orders o
        ON o.customerNumber = c.customerNumber
        INNER JOIN orderdetails od
        ON od.orderNumber = o.orderNumber
        INNER JOIN products p
        ON p.productCode = od.productCode
        GROUP BY e.firstName, e.lastName
        ORDER BY num_products_sold DESC
        """
df = pd.DataFrame(c.execute(query).fetchall())
df.columns = [x[0] for x in c.description]
df

,firstName,lastName,num_products_sold
0,Gerard,Hernandez,14231
1,Leslie,Jennings,11854
2,Pamela,Castillo,9290
3,Larry,Bott,8205
4,Barry,Jones,7486
5,George,Vanauf,7423
6,Peter,Marsh,6632
7,Andy,Fixter,6246
8,Loui,Bondur,6186
9,Steve,Patterson,5561


## Summary

Congrats! You now know how to use join statements, along with leveraging your foreign keys knowledge!